In [1]:
# Imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
# Initialize OpenAI API
openai = OpenAI()

In [5]:
# Website class
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given URL using the BeautifulSoup library.
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# System prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# Function to generate user prompt
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
# Function to prepare messages for OpenAI API
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
# Summarize a single website
def summarize(url):
    try:
        website = Website(url)
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages_for(website)
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error summarizing {url}: {e}"

In [10]:
# Display summary for a single URL
def display_summary(url):
    summary = summarize(url)
    display(Markdown(f"## Summary for {url}\n\n{summary}"))


In [11]:
# Summarize multiple URLs
def display_summaries(urls):
    for url in urls:
        display_summary(url)

In [12]:
# List of URLs
urls = [
    "https://edwarddonner.com",
    "https://cnn.com"
]

In [13]:
# Run the summarization
display_summaries(urls)

## Summary for https://edwarddonner.com

# Summary of Edward Donner's Website

The website features Ed Donner, a developer and AI enthusiast, who expresses a keen interest in coding and experimenting with large language models (LLMs). He is the co-founder and CTO of Nebula.io, which aims to leverage AI in talent discovery and management. Previously, he founded an AI startup, untapt, which was acquired in 2021. The site includes a section called "Outsmart," presenting an LLM arena that focuses on diplomatic challenges and strategic gameplay.

## Recent Posts
- **November 13, 2024**: Resources for mastering AI and LLM engineering.
- **October 16, 2024**: Resources for transitioning from a software engineer to an AI data scientist.
- **August 6, 2024**: Overview of the Outsmart LLM arena.
- **June 26, 2024**: Guidance on selecting the right LLM with recommended toolkits and resources. 

Ed encourages connections and collaborations related to AI and LLMs.

## Summary for https://cnn.com

# CNN News Summary

CNN is a global news platform providing the latest headlines and reports across various topics, including US and world news, politics, business, health, entertainment, and technology. The site features extensive coverage of current events, including:

- **Syria**: The regime of Bashar al-Assad has reportedly fallen after decades, with swift victories by Syrian rebels. Live updates indicate ongoing strikes in Damascus and celebratory demonstrations following the regime's collapse.
- **US Politics**: Former President Trump faces increasing scrutiny and discusses potential actions related to previous immigration policies and the January 6th incident, calling for jail time for specific committee members.
- **International Affairs**: South Korea has imposed travel restrictions on its president amid an expanding insurrection investigation. 

Additionally, the site explores human-interest stories, including a feature on CNN's Hero of the Year who provides fostering services for dogs while their owners seek addiction treatment. Other significant reports include discussions on health issues, celebrity news, and global business matters. 

Overall, CNN serves as a comprehensive source for breaking news and in-depth analysis across numerous subjects and events happening globally.